# Example use case of PynPoint-IFS

# Import statements and pipeline initialisation

In [ ]:
import os
os.environ['OMP_NUM_THREADS'] = '30'
import numpy as np
import pynpoint as pp
from tqdm import tqdm
import matplotlib.pyplot as plt
from astropy.modeling import models, fitting
import pandas as pd
from astropy.io import fits
from glob import glob
import sys
from pathlib import Path
from PyAstronomy.pyasl import crosscorrRV,dopplerShift
from scipy.ndimage import gaussian_filter
from scipy.signal import medfilt,savgol_filter
import spectres
from skimage.registration import phase_cross_correlation
from datetime import datetime,timedelta
from mpl_toolkits.axes_grid1 import make_axes_locatable
from photutils.centroids import centroid_quadratic 

In [ ]:
from pynpoint_ifs.ifuframeselection import SelectWavelengthRangeModule,AutomaticallySelectWavelengthRangeModule
from pynpoint_ifs.ifubadpixel import NanFilterModule,OutlierCorrectionModule
from pynpoint_ifs.ifucentering import IFUAlignCubesModule,FitCenterCustomModule
from pynpoint_ifs.ifupsfpreparation import IFUStellarSpectrumModule,IFUWavelengthCalibrationModule,IFUWavelengthCorrectionModule,IFUSDIpreparationModule,IFUTelluricsWavelengthCalibrationModule
from pynpoint_ifs.ifupsfsubtraction import IFUContinuumRemovalModule,IFUPSFSubtractionModuleCugno,IFUPCAPSFSubtractionModule,IFUPCAModule
from pynpoint_ifs.ifuresizing import FoldingModule,FinerGridInterpolationModule
from pynpoint_ifs.ifustacksubset import CrossCorrelationPreparationModule,StackCubesModule,ApertureCombineModule
from pynpoint_ifs.ifucrosscorrelation import CrossCorrelationModule
from pynpoint_ifs.ifuresizing import UnfoldingModule,UpSampleModule
from pynpoint_ifs.ifufitswriting import IFUFitsWritingModule
from pynpoint_ifs.ifufluxcalibration import IFUSpectrumExtractionModule
from pynpoint_ifs.ifufakeplanetinjection import IFUFakePlanetInjectionModule
from pynpoint_ifs.ifuprocessingfunctions import do_PCA_sub,do_derotate_shift
from pynpoint_ifs.ifuprocessing import IFUPostProcessingModule
from pynpoint_ifs.ifu_utils import *
from pynpoint_ifs.ifu_utils_hidden import *
from pynpoint_ifs.ifu_plotting import *
from pynpoint_ifs.ifu_pipeline import run_preprocessing

In [ ]:
# where the data is stored
data_dir = 'absolute/path/to/data/folder/'
# define where to save the data produced by the steps of the pipeline
working_data = 'absolute/path/to/working/folder/'
# path to where the plots and finished products should be saved
save_dir = 'absolute/path/to/save/folder/'

In [ ]:
pipeline = pp.Pypeline(working_place_in=working_data,
                    input_place_in=data_dir,
                    output_place_in=data_dir)

In [ ]:
# change the configuration file to the right pixel scale, nb of CPUs, and memory size
# change_config(pipeline,pixscale=0.0125,cpu=20,memory=750)

In [ ]:
# these are outputs from SpyFFIER
# here newxcorr_spline_fov-linear_product_g_* correspond to groups of SKY-OBJECT frames
planet_files = list(map(str,Path(data_dir).glob('science_ifu_jitter/wvlcorr_product_g_*/eris_ifu_jitter_dar_cube_*_corr_wavemap_*fits')))

In [ ]:
sequence_files = {
    'planet':planet_files
}

In [ ]:
which = 'planet'

# Preprocessing steps

In [ ]:
run_preprocessing(pipeline,sequence_files,run_which=which,bad_pixel_corr=False,crop = 2,outlier_sigma=30)

In [ ]:
pipeline.set_attribute(data_tag = 'raw_'+ which + '_wvlsel_nancorr_crop_outlier',attr_name='PARANG',attr_value = np.zeros((lencube)),static=False)

# Measure centroid of the star

In [ ]:
datacube,wavelength = get_cubes(pipeline,'raw_'+which+'_wvlsel_nancorr_crop_outlier','wavelength_'+which)

In [ ]:
image_cube = np.mean(datacube,axis=1)

In [ ]:
centroid_pointsource(pipeline,image_cube,filter_sigma=1,plot=True,save=True,save_fit_tag=which + '_cubeposition')

In [ ]:
data = pipeline.get_data(which + '_cubeposition')

In [ ]:
planet_position = np.array([-225.199,331.019]) # approximate position of the planet relative to the star

In [ ]:
pixscale=pipeline.get_attribute('raw_'+which+'_wvlsel_nancorr_crop_outlier',attr_name='PIXSCALE')

In [ ]:
planet_position_pix = np.array([-planet_position[0],planet_position[1]])/1e3/pixscale

In [ ]:
lencube,lenwvl,lenx,leny=np.shape(datacube)

In [ ]:
data = pipeline.get_data(which + '_cubeposition')
plt.plot(data[:,0])
plt.plot(data[:,2])
plt.plot(data[:,0] + planet_position_pix[0])
plt.plot(data[:,2] + planet_position_pix[1])
plt.axhline(lenx/2 - 1)
plt.axhline(-(lenx/2 - 1))

# Frame selection

In [ ]:
remove_mask = np.logical_or(data[:,0] + planet_position_pix[0] > lenx/2 - 1,data[:,2] + planet_position_pix[1] > leny/2 - 1)
remove_frames = np.arange(len(data))[remove_mask]
remove_frames_inds = np.array([np.arange(lenwvl*i,lenwvl*(i+1)) for i in remove_frames]).flatten()

In [ ]:
module = pp.RemoveFramesModule(
    name_in='remove_outofFOV', 
    image_in_tag='raw_'+ which + '_wvlsel_nancorr_crop_outlier', 
    selected_out_tag='raw_'+ which + '_wvlsel_nancorr_crop_outlier_sel', 
    removed_out_tag = 'None', 
    frames = remove_frames_inds)
pipeline.add_module(module)
pipeline.run_module('remove_outofFOV')

nframes = pipeline.get_attribute('raw_'+ which + '_wvlsel_nancorr_crop_outlier',attr_name='NFRAMES',static=False)
new_nframes = nframes[~np.isin(np.arange(len(nframes)), remove_frames)]
pipeline.set_attribute(data_tag = 'raw_'+ which + '_wvlsel_nancorr_crop_outlier_sel',attr_name='NFRAMES',attr_value = new_nframes,static=False)
files = pipeline.get_attribute('raw_'+ which + '_wvlsel_nancorr_crop_outlier',attr_name='FILES',static=False)
new_files = files[~np.isin(np.arange(len(files)), remove_frames)]
pipeline.set_attribute(data_tag = 'raw_'+ which + '_wvlsel_nancorr_crop_outlier_sel',attr_name='FILES',attr_value = new_files,static=False)

In [ ]:
datacube,wavelength = get_cubes(pipeline,'raw_'+which+'_wvlsel_nancorr_crop_outlier_sel','wavelength_'+which)

In [ ]:
image_cube = np.mean(datacube,axis=1)
centroid_pointsource(pipeline,image_cube,filter_sigma=1,plot=True,save=True,save_fit_tag=which + '_sel_cubeposition')

# Test PSF subtraction with spectral PCA using different numbers of PCs

In [ ]:
pca_numbers = np.logspace(np.log10(40),np.log10(500),5,dtype=int)

In [ ]:
template_path = '/home/ipa/quanz/shared/eris/P112_atmospheres/planet_templates/grid_temperature/templates_T_1200/'
wavelength = pipeline.get_data('wavelength_' + which)
wlen_mol,flux_mol = load_custom_templates(glob(template_path + '*csv'),wavelength,sigma=20)
molecules = list(wlen_mol.keys())
rv_range,drv=300,1

In [ ]:
molecules = ['H2O','CO','CO2','CH4','chem_equ']

In [ ]:
for pca_i,pca in enumerate(pca_numbers):
    # PCA
    module = IFUPCAPSFSubtractionModule(
        name_in = f'PCA_sub_{pca}',
        image_in_tag = 'raw_'+ which + '_wvlsel_nancorr_crop_outlier',
        wv_in_tag = 'wavelength_' + which,
        image_out_tag = 'HRSDI_PCA',
        pca_number = int(pca),
        method='single'
        )
    pipeline.add_module(module)
    pipeline.run_module(f'PCA_sub_{pca}')
    # frame combining
    module = CrossCorrelationPreparationModule(
        name_in=f'combine_HRSDI_PCA_{pca}',
        image_in_tag='HRSDI_PCA',
        shift_cubes_in_tag=which + '_cubeposition_relative',
        image_out_tag='HRSDI_PCA_prep',
        mask_out_tag='mask',
        shift=True,
        rotate=False,
        stack=True)
    pipeline.add_module(module)
    pipeline.run_module(f'combine_HRSDI_PCA_{pca}')
    # cross-correlation
    for mol_i in range(len(molecules)):
        molec = molecules[mol_i]
        module = CrossCorrelationModule(
            name_in = f'combine_HRSDI_PCA_{pca}_{molec}',
            RV = rv_range,
            dRV = drv,
            range_CCF_RV = 100.,
            data_wv_in_tag = 'wavelength_' + which,
            model_wv = wlen_mol[molec],
            model_abs = flux_mol[molec],
            image_in_tag = 'HRSDI_PCA_prep',
            SNR_cube_out_tag = f'snr_{pca}_{molec}_' + which,
            CC_cube_out_tag = f'CC_{pca}_{molec}_' + which,
            RV_data_out_tag = f'RV_{pca}_{molec}_' + which,
            cpus=20
            )
        pipeline.add_module(module)
        pipeline.run_module(f'combine_HRSDI_PCA_{pca}_{molec}')

In [ ]:
rv = np.arange(-rv_range,rv_range+0.5*drv,drv)
rv_planet = 0
col_nb = len(molecules)+1
combined_cc = {}
combined_snr = {}
for pca_i,pca in enumerate(pca_numbers):
    fig,axes = plt.subplots(figsize=(3*(len(molecules)+1),5),ncols=col_nb)
    cc_mol={}
    for mol_i in range(len(molecules)):
        molec = molecules[mol_i]
        cc_mol[molec] = pipeline.get_data(f'CC_{pca}_{molec}_' + which)[0]
        # snr = snr_map(rv,cc_mol[molec],signal_range=(-20,20),std_interval = 100)
        # snr = pipeline.get_data(f'snr_{pca}_{molec}')[0]
        snr = cc_mol[molec][len(rv)//2]
        # smooth = gaussian_filter(snr,0.75)
        axes[mol_i].imshow(snr,origin='lower')
    CC_tot = np.product([cc_mol[key] for key in molecules],axis=0)
    # snr_from_CC = snr_map(rv,CC_tot,signal_range=(-20,20),std_interval = 100)
    snr_from_CC = CC_tot[len(rv)//2 + int(rv_planet/drv)]
    combined_cc[pca] = CC_tot
    combined_snr[pca] = snr_from_CC
    # smooth = gaussian_filter(snr_from_CC,0.75)
    axes[col_nb-1].imshow(snr_from_CC,origin='lower')
    for i in range(col_nb):
        if i < len(molecules):
            mol_title = molecules[i]
        else:
            mol_title = 'all'
        axes[i].set_title('PCA: %i, Mol: %s' % (pca,mol_title))
        axes[i].scatter(x=39,y=44,color='w',s=2)
        # axes[i].scatter(x=lenx/2 + mean_pos_planet[0], y= lenx/2 + mean_pos_planet[1],s=1,color='r',marker='x')
        # axes[i].scatter(x= lenx/2  + mean_pos_planet[0] - planet_pos_px[0], y= lenx/2 + mean_pos_planet[1] - planet_pos_px[1],s=50,color='orange',marker='*')
    plt.show()

# Select the nb of PCs and continue

In [ ]:
pca = 250

In [ ]:
module = IFUPCAPSFSubtractionModule(
    name_in = f'PCA_sub_{pca}',
    image_in_tag = 'raw_'+ which + '_wvlsel_nancorr_crop_outlier_sel',
    wv_in_tag = 'wavelength_' + which,
    image_out_tag = f'HRSDI_PCA_{pca}',
    pca_number = int(pca),
    method='single'
    )
pipeline.add_module(module)
pipeline.run_module(f'PCA_sub_{pca}')

In [ ]:
# frame combining
module = CrossCorrelationPreparationModule(
    name_in=f'combine_HRSDI_PCA_{pca}',
    image_in_tag=f'HRSDI_PCA_{pca}',
    shift_cubes_in_tag=which + '_sel_cubeposition_relative',
    image_out_tag=f'HRSDI_PCA_{pca}_prep',
    mask_out_tag='mask',
    shift=True,
    rotate=False,
    stack=True)
pipeline.add_module(module)
pipeline.run_module(f'combine_HRSDI_PCA_{pca}')

In [ ]:
module = ApertureCombineModule(
    name_in = 'aperture_combine',
    image_in_tag = f'HRSDI_PCA_{pca}_prep',
    image_out_tag = f'HRSDI_PCA_{pca}_prep_apertures',
    aperture_radius = 1.7,
    cpus=10
)
pipeline.add_module(module)
pipeline.run_module('aperture_combine')

In [ ]:
template_path = '/home/ipa/quanz/shared/eris/P112_atmospheres/planet_templates/grid_temperature/templates_T_1200/'
wavelength = pipeline.get_data('wavelength_' + which)
wlen_mol,flux_mol = load_custom_templates(glob(template_path + '*csv'),wavelength,sigma=20)
molecules = list(wlen_mol.keys())
rv_range,drv=300,1

In [ ]:
molecules = list(wlen_mol.keys())

In [ ]:
# cross-correlation
for mol_i in range(len(molecules)):
    molec = molecules[mol_i]
    module = CrossCorrelationModule(
        name_in = f'combine_HRSDI_PCA_apertures_{pca}_{molec}',
        RV = rv_range,
        dRV = drv,
        range_CCF_RV = 100.,
        data_wv_in_tag = 'wavelength_' + which,
        model_wv = wlen_mol[molec],
        model_abs = flux_mol[molec],
        image_in_tag = f'HRSDI_PCA_{pca}_prep_apertures',
        SNR_cube_out_tag = f'snr_apertures_{pca}_{molec}',
        CC_cube_out_tag = f'CC_apertures_{pca}_{molec}',
        RV_data_out_tag = f'RV_apertures_{pca}_{molec}',
        cpus=20
        )
    pipeline.add_module(module)
    pipeline.run_module(f'combine_HRSDI_PCA_apertures_{pca}_{molec}')

In [ ]:
rv = np.arange(-rv_range,rv_range+0.5*drv,drv)
rv_planet = 0
col_nb = len(molecules)+1
fig,axes = plt.subplots(figsize=(3*(len(molecules)+1),5),ncols=col_nb)
cc_mol={}
for mol_i in range(len(molecules)):
    molec = molecules[mol_i]
    cc_mol[molec] = pipeline.get_data(f'CC_apertures_{pca}_{molec}')[0]
    # snr = snr_map(rv,cc_mol[molec],signal_range=(-20,20),std_interval = 100)
    # snr = pipeline.get_data(f'snr_{pca}_{molec}')[0]
    snr = cc_mol[molec][len(rv)//2]
    # smooth = gaussian_filter(snr,0.75)
    axes[mol_i].imshow(snr,origin='lower')
CC_tot = np.product([cc_mol[key] for key in molecules],axis=0)
# snr_from_CC = snr_map(rv,CC_tot,signal_range=(-20,20),std_interval = 100)
snr_from_CC = CC_tot[len(rv)//2 + int(rv_planet/drv)]
combined_cc = {}
combined_snr = {}
combined_cc[pca] = CC_tot
combined_snr[pca] = snr_from_CC
# smooth = gaussian_filter(snr_from_CC,0.75)
axes[col_nb-1].imshow(snr_from_CC,origin='lower')
for i in range(col_nb):
    if i < len(molecules):
        mol_title = molecules[i]
    else:
        mol_title = 'all'
    axes[i].set_title('PCA: %i, Mol: %s' % (pca,mol_title))
    axes[i].scatter(x=39,y=44,color='w',s=2)
    # axes[i].scatter(x=lenx/2 + mean_pos_planet[0], y= lenx/2 + mean_pos_planet[1],s=1,color='r',marker='x')
    # axes[i].scatter(x= lenx/2  + mean_pos_planet[0] - planet_pos_px[0], y= lenx/2 + mean_pos_planet[1] - planet_pos_px[1],s=50,color='orange',marker='*')
plt.show()

# Fit position of planet in molecular maps

In [ ]:
molecules_considered = ['H2O','CO','H2O+CO','chem_equ']

In [ ]:
planet_position_mol = {}
for mol_i,mol in enumerate(molecules_considered):
    molec = molecules_considered[mol_i]
    print(f'CC_apertures_{pca}_{molec}')
    ccf = pipeline.get_data(f'CC_apertures_{pca}_{molec}')[0,:]
    snr = ccf[len(rv)//2] # snr_map(rv,ccf,signal_range=(-20,20),std_interval = 100)
    # snr = pipeline.get_data(f'snr_apertures_{pca}_{mol}')[0,crop:-crop,crop:-crop]
    # smooth = gaussian_filter(snr,0.75)
    mask = np.isnan(snr)
    image = snr
    lenx,leny=np.shape(image)
    image_f = image-np.nanmedian(image)
    
    x1, y1 = centroid_quadratic(image_f,mask=mask)
    planet_position_mol[mol] = [x1 - (lenx-1)/2,y1 - (leny-1)/2]
    
    plt.figure()
    plt.imshow(snr,origin='lower')
    plt.scatter(x=x1,y=y1)
    plt.title(mol)
    # plt.colorbar()
    plt.show()
    
    plt.figure()
    plt.plot(rv,ccf[:,int(y1),int(x1)])
    plt.show()

In [ ]:
planet_position = np.mean([planet_position_mol[mol] for mol in planet_position_mol],axis=0)

In [ ]:
fitparams = np.zeros((1,14))
fitparams[0,0] = planet_position[0]
fitparams[0,2] = planet_position[1]

In [ ]:
out_port = pp.core.dataio.OutputPort('planet_position', data_storage_in=pipeline.m_data_storage)
out_port.set_all(fitparams)
out_port.close_port()

In [ ]:
rel_pos = pipeline.get_data(which + '_sel_cubeposition_relative')

In [ ]:
absolute_position_planet = fitparams + rel_pos

In [ ]:
out_port = pp.core.dataio.OutputPort('planet_cubeposition_abs', data_storage_in=pipeline.m_data_storage)
out_port.set_all(absolute_position_planet)
out_port.close_port()

In [ ]:
planet_position = pipeline.get_data('planet_position')[0,(0,2)]

In [ ]:
rv_axis = pipeline.get_data(f'RV_apertures_{pca}_{molec}')

In [ ]:
for mol_i,mol in enumerate(molecules_considered):
    molec = molecules_considered[mol_i]
    ccf = pipeline.get_data(f'CC_apertures_{pca}_{molec}')[0]
    lenrv,lenx,leny = np.shape(ccf)
    planet_pos_det = (planet_position + (lenx-1)/2).astype(int)
    std_ccf = np.nanstd(ccf)
    ccf_planet = ccf[:,planet_pos_det[1],planet_pos_det[0]]/std_ccf
    plt.plot(rv_axis,ccf_planet,label=molec)
plt.legend()

# Show molecular map

## PSF picture

In [ ]:
datacubes,wavelength = get_cubes(pipeline,'raw_'+ which + '_wvlsel_nancorr_crop_outlier_sel','wavelength_' + which)

In [ ]:
lencube,lenwvl,lenx,leny=np.shape(datacubes)

In [ ]:
upscale = 3
cubeposition = pipeline.get_data(which + '_sel_cubeposition_relative')[:,(0,2)]

In [ ]:
shifted_images = nice_psf_image(datacubes,upscale,cubeposition)

In [ ]:
image_mean = np.nanmean(shifted_images[:],axis=0)
img_log = np.log10(image_mean)

In [ ]:
add_term=0

In [ ]:
plt.imshow(img_log,vmin=np.percentile(img_log,5),vmax=np.percentile(img_log,99.999),cmap='afmhot',origin='lower')

## Molecular maps

In [ ]:
fontsize=12
crop_x = 15
crop_y = 10

In [ ]:
molecules_considered = ['H2O', 'CO', 'CH4', 'CO2', 'chem_equ','CO36','HCN','NH3','FeH','TiO','VO']
molecules_names = {mol:nice_name(mol) for mol in molecules_considered}

In [ ]:
wavelength = pipeline.get_data('wavelength_' + which)
lod = np.mean(wavelength)*1e-6/8.2/np.pi*180*60*60
pixscale = pipeline.get_attribute('raw_'+ which + '_wvlsel_nancorr_crop_outlier_sel',attr_name='PIXSCALE')
lod_px = lod/pixscale

In [ ]:
circle_width = lod_px
width_annulus = lod_px

In [ ]:
# position of planet and star in the pictures
planet_position = pipeline.get_data('planet_position')[0,(0,2)]
fitparams_star = pipeline.get_data(which + '_sel_cubeposition')[0,(0,2)]
fitparams_star_rel = pipeline.get_data(which + '_sel_cubeposition_relative')[0,(0,2)]
star_pos_mean = fitparams_star-fitparams_star_rel

In [ ]:
# rv of the detected planet
rv_planet=0

In [ ]:
ncols = 3
nrows=int(np.ceil((len(molecules_considered)+1)/ncols))
fig,axes = plt.subplots(figsize=(2.5*ncols,2.2*nrows),ncols=ncols,nrows=nrows)

# PSF
image_mean = np.nanmean(shifted_images,axis=0)
star_pos = np.array([upscale*((lenx)/2  + star_pos_mean[0]), upscale*((lenx)/2 + star_pos_mean[1])])
planet_pos = np.array([upscale*((lenx-1)/2 + planet_position[0]), upscale*((lenx-1)/2 + planet_position[1])])
ticks_x_labels,tick_x_pos,ticks_x_labels_minor,tick_x_pos_minor,ticks_y_labels,tick_y_pos,ticks_y_labels_minor,tick_y_pos_minor=get_ticks(image_mean,star_pos,pixscale=pixscale/3,tick_sep=0.1,minor_tick_sep = 0.05)
tick_params = [ticks_x_labels,tick_x_pos,ticks_x_labels_minor,tick_x_pos_minor,ticks_y_labels,tick_y_pos,ticks_y_labels_minor,tick_y_pos_minor]
make_PSF_plot(ax=axes[0,0],image=image_mean,crop=0,tick_params = tick_params,fontsize=fontsize,star_pos = star_pos,title='Intensity')
plot_circle(axes[0,0],planet_pos,upscale*circle_width)
axes[0,0].set_xlim((0,len(image_mean)-crop_x*upscale))
axes[0,0].set_ylim((0,len(image_mean)-crop_y*upscale))

# molecular maps
for mol_i,mol in enumerate(molecules_considered):
    cc = pipeline.get_data(f'CC_apertures_{pca}_{mol}')[0]
    drv = pipeline.get_data(f'RV_apertures_{pca}_{mol}')
    # cc = pipeline.get_data(f'CC_{pca}_{mol}_' + which)[0]
    # drv = pipeline.get_data(f'RV_{pca}_{mol}_' + which)
    #cc = pipeline.get_data(f'CC_NOapertures_{pca}_{mol}_wb')[0]
    #drv = pipeline.get_data(f'RV_NOapertures_{pca}_{mol}_wb')
    axi = (mol_i+1) // ncols
    axj = (mol_i+1) % ncols
    star_pos = np.array([((lenx)/2  + star_pos_mean[0]), ((lenx)/2 + star_pos_mean[1])])
    planet_pos = np.array([(lenx-1)/2 + planet_position[0], (lenx-1)/2 + planet_position[1]])
    
    if mol in ['H2O','CO','H2O+CO','chem_equ']:
        snr_ttest,bins_vals,bins_pos,popt,ccf_pl_val = calculate_molmap_metric(cc,drv,planet_pos,star_pos,rv_planet,mask_width=width_annulus,n_bins=20,method='image')

    
    ticks_x_labels,tick_x_pos,ticks_x_labels_minor,tick_x_pos_minor,ticks_y_labels,tick_y_pos,ticks_y_labels_minor,tick_y_pos_minor=get_ticks(cc[0,:,:],star_pos,pixscale=pixscale,tick_sep=0.1,minor_tick_sep = 0.05)
    tick_params = [ticks_x_labels,tick_x_pos,ticks_x_labels_minor,tick_x_pos_minor,ticks_y_labels,tick_y_pos,ticks_y_labels_minor,tick_y_pos_minor]
    
    make_ccf_plot(axes[axi,axj],cc,drv,rv_planet,planet_pos,star_pos,vmin=-1,vmax=5)
    plot_circle(axes[axi,axj],planet_pos,circle_width)
    # if axj == 2:
    #     axes[axi,axj].yaxis.set_label_position("right")
    #     axes[axi,axj].yaxis.tick_right()
    make_labels(axes[axi,axj],tick_params=tick_params,img=cc[0],fontsize=fontsize,title=molecules_names[molecules_considered[mol_i]])
    if axj == 1 and axi == 0:
        axes[axi,axj].set_ylabel('')
    
    if mol in ['H2O','CO','H2O+CO','chem_equ']:
        annotate_snr(axes[axi,axj],planet_pos + np.array([-20,0]),snr_ttest,fontsize=12)
    axes[axi,axj].set_xlim((0,len(cc[0])-crop_x))
    axes[axi,axj].set_ylim((0,len(cc[0])-crop_y))

plt.tight_layout()
plt.subplots_adjust(left=0.,
                    right=1., 
                    bottom=0., 
                    top=1., 
                    wspace=0.3, 
                    hspace=0.4)
fig.savefig(save_dir + 'HR8799e_detection_molmaps.png',dpi=300,bbox_inches='tight')
plt.show()

## Plot detection metrics

In [ ]:
detected_models = ['H2O','CO','chem_equ']

In [ ]:
# plot for appendix to explain metric
crop=1
fig,axes=plt.subplots(nrows=2,ncols=len(detected_models),figsize=(2.45*len(detected_models),2.2*2))
axes= np.transpose(axes)
snr_tests_vals = {}
for mol_i,mol in enumerate(detected_models):

    cc = pipeline.get_data(f'CC_apertures_{pca}_{mol}')[0]
    drv = pipeline.get_data(f'RV_apertures_{pca}_{mol}')
    star_pos = np.array([((lenx)/2  + star_pos_mean[0]), ((lenx)/2 + star_pos_mean[1])])
    planet_pos = np.array([(lenx-1)/2 + planet_position[0], (lenx-1)/2 + planet_position[1]])
    print(planet_pos)
    dist_pl_st=np.linalg.norm(planet_pos-star_pos)
    if mol in ['H2O','CO','H2O+CO','chem_equ']:
        snr_ttest,bins_vals,bins_pos,popt,ccf_pl_val = calculate_molmap_metric(cc,drv,planet_pos,star_pos,rv_planet,mask_width=width_annulus,n_bins=20,crop=crop,method='image')
    snr_tests_vals[mol] = snr_ttest
    
    ticks_x_labels,tick_x_pos,ticks_x_labels_minor,tick_x_pos_minor,ticks_y_labels,tick_y_pos,ticks_y_labels_minor,tick_y_pos_minor=get_ticks(cc[0,:,:],star_pos,pixscale=pixscale,tick_sep=0.1,minor_tick_sep = 0.05)
    tick_params = [ticks_x_labels,tick_x_pos,ticks_x_labels_minor,tick_x_pos_minor,ticks_y_labels,tick_y_pos,ticks_y_labels_minor,tick_y_pos_minor]
    
    make_ccf_plot(axes[mol_i,0],cc,drv,rv_planet,planet_pos,star_pos,vmin=-1,vmax=6)
    plot_circle(axes[mol_i,0],planet_pos,width_annulus)
    plot_circle(axes[mol_i,0],star_pos,dist_pl_st + dist_pl_st)
    plot_circle(axes[mol_i,0],star_pos,dist_pl_st - dist_pl_st)
    make_labels(axes[mol_i,0],tick_params=tick_params,img=cc[0],fontsize=fontsize,title=molecules_names[mol])
    if mol in ['H2O','CO','H2O+CO','chem_equ']:
        annotate_snr(axes[mol_i,0],planet_pos + np.array([0,7]),snr_ttest,fontsize=fontsize)
    axes[mol_i,0].set_xlim((crop,len(cc[0])-crop))
    axes[mol_i,0].set_ylim((crop,len(cc[0])-crop))
    
    # histogram
    
    axes[mol_i,1].stairs(bins_vals,bins_pos,fill=True)
    x=np.linspace(-20,20,500)
    y = gaussian(x,popt[0],popt[1],popt[2])
    axes[mol_i,1].plot(x,y,color='k')
    axes[mol_i,1].axvline(ccf_pl_val,color='orange')

    textstr = '\n'.join((
        r'$\mu=%.2f$' % (popt[0], ),
        r'$\sigma=%.2f$' % (np.abs(popt[1]), ),
        r'$C_{\mathrm{p}}=%.2f$' % (ccf_pl_val, )))
    props = dict(boxstyle='round', facecolor='w', alpha=0.5)
    axes[mol_i,1].text(0.45, 0.95, textstr, transform=axes[mol_i,1].transAxes, fontsize=fontsize-2,
        verticalalignment='top',horizontalalignment='left', bbox=props)
    
    title_name = molecules_names[mol]
    axes[mol_i,1].set_title(title_name,fontsize=fontsize)
    ticks=np.arange(-4,(int(ccf_pl_val)//2)*2+2)
    axes[mol_i,1].set_xticks(ticks=ticks,minor=True)
    axes[mol_i,1].set_xticks(ticks=ticks[::2],minor=False)
    axes[mol_i,1].tick_params(which='both',axis='both',labelsize=fontsize-4)
    axes[mol_i,1].set_xlabel('Normalised CCF values',fontsize=fontsize-2)
    axes[mol_i,1].set_ylabel('Occurence rate',fontsize=fontsize-2)
    axes[mol_i,1].set_xlim((-4,ccf_pl_val+1))
    axes[mol_i,1].set_box_aspect(0.95)

# finish layout
plt.tight_layout()

plt.subplots_adjust(left=0.,
                    right=1., 
                    bottom=0., 
                    top=1., 
                    wspace=0.3, 
                    hspace=0.35)
fig.savefig(save_dir + 'HR8799e_detection_metric.png',dpi=300,bbox_inches='tight')
plt.show()

# Extract spectra

In [ ]:
pca=250

In [ ]:
# get spectrum of star
module = IFUSpectrumExtractionModule(
    name_in = 'extract_spectrum_star',
    image_in_tag = 'raw_'+ which + '_wvlsel_nancorr_crop_outlier_sel',
    obj_position_in_tag = which + '_sel_cubeposition',
    obj_spectra_out_tag = 'star' + '_spectra_obj',
    bk_spectra_out_tag = 'star' + '_spectra_bk',
    aperture_obj_radius = 3*lod_px,
    aperture_bk_radius = 3,
    aperture_bk_nb = 0,
    aperture_bk_dist = 11.,
    aperture_bk_angle = 0.,
    aperture_bk_position = 'around',
    plot = True
)
pipeline.add_module(module)
pipeline.run_module('extract_spectrum_star')

In [ ]:
# spectrum of planet
module = IFUSpectrumExtractionModule(
    name_in = 'extract_spectrum_planet',
    image_in_tag = f'HRSDI_PCA_{pca}',
    obj_position_in_tag = 'planet_cubeposition_abs',
    obj_spectra_out_tag = which + '_spectra_obj',
    bk_spectra_out_tag = which + '_spectra_bk',
    aperture_obj_radius = 1.7,
    aperture_bk_radius = 1.7,
    aperture_bk_nb = 10,
    aperture_bk_dist = 11.,
    aperture_bk_angle = 0.,
    aperture_bk_position = 'around',
    plot = False
)
pipeline.add_module(module)
pipeline.run_module('extract_spectrum_planet')

In [ ]:
# spectrum of planet after combination
module = IFUSpectrumExtractionModule(
    name_in = 'extract_spectrum_planet_prep',
    image_in_tag = f'HRSDI_PCA_{pca}_prep',
    obj_position_in_tag = 'planet_position',
    obj_spectra_out_tag = which + '_spectra_obj_prep',
    bk_spectra_out_tag = which + '_spectra_bk_prep',
    aperture_obj_radius = 1.7,
    aperture_bk_radius = 1.7,
    aperture_bk_nb = 10,
    aperture_bk_dist = 11.,
    aperture_bk_angle = 0.,
    aperture_bk_position = 'around',
    plot = True
)
pipeline.add_module(module)
pipeline.run_module('extract_spectrum_planet_prep')

In [ ]:
# get spectrum of star at the position of the planet
module = IFUSpectrumExtractionModule(
    name_in = 'extract_spectrum_star_planet_pos',
    image_in_tag = 'raw_'+ which + '_wvlsel_nancorr_crop_outlier_sel',
    obj_position_in_tag = 'planet_cubeposition_abs',
    obj_spectra_out_tag = 'star_spectra_obj_planet_pos',
    bk_spectra_out_tag = 'star_spectra_bk_planet_pos',
    aperture_obj_radius = 1.7,
    aperture_bk_radius = 1.7,
    aperture_bk_nb = 0,
    aperture_bk_dist = 11.,
    aperture_bk_angle = 0.,
    aperture_bk_position = 'around',
    plot = True
)
pipeline.add_module(module)
pipeline.run_module('extract_spectrum_star_planet_pos')

In [ ]:
wavelength = pipeline.get_data('wavelength_' + which)
np.savetxt(save_dir + 'wavelength.txt',wavelength)

spectra_star = pipeline.get_data('star_spectra_obj')
np.savetxt(save_dir + 'spectra_star.txt',spectra_star)

spectra_planet = pipeline.get_data('planet_spectra_obj')
np.savetxt(save_dir + 'spectra_planet.txt',spectra_planet)

spectra_star = pipeline.get_data('star_spectra_obj_planet_pos')
np.savetxt(save_dir + 'spectra_star_planet_pos.txt',spectra_star)

In [ ]:
# save timestamp of each frame
dateobs = pipeline.get_attribute('raw_'+ which + '_wvlsel_nancorr_crop_outlier_sel',attr_name='DATE',static=False).astype(str)
dit = pipeline.get_attribute('raw_'+ which + '_wvlsel_nancorr_crop_outlier_sel',attr_name='DIT',static=True)
delta_dit = timedelta(seconds=dit/2)
time_frame = np.array(list(map(lambda x: datetime.fromtimestamp(np.round((datetime.fromisoformat(x)-delta_dit).timestamp())).isoformat(),dateobs))).astype(str)
np.savetxt(save_dir + 'datetime.txt',time_frame,fmt='%s')